In [48]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.coordinates import SkyCoord
import warnings
import pickle
warnings.filterwarnings('ignore')

Define observing constraints: 

In [49]:
# Equilibrium temperatures
teq_min = 0 * u.K
teq_max = 1000 * u.K

rad_min = 0 * u.R_earth
rad_max = 10 * u.R_earth
gaia_mag_limit = 15

Download NASA Exoplanet Archive table from the web via astroquery:

In [50]:
table = NasaExoplanetArchive.get_confirmed_planets_table(all_columns=True)
transiting = table[(table['pl_tranflag'].data).astype(bool)]
pickle.dump( transiting, open( "table.p", "wb" ) )

Compute equilibrium temperatures for zero albedo and the (relative) scale height: 

In [34]:
rp = transiting['pl_radj']
equilibrium_temperature = (transiting['st_teff'] * np.sqrt(transiting['st_rad'] / 2 / transiting['pl_orbsmax'])).decompose()
g = transiting['gaia_gmag'].quantity
transiting['teq'] = equilibrium_temperature

mean_molecular_mass = 6 * u.u#Assuming a default mean molecular weight. Unknown for almost all targets.
gp = const.G * 10*u.earthMass / rp**2#Assuming a default mass of 10Me. Most of our targets have no masses.
transiting['H'] = ((const.k_B * equilibrium_temperature) / (mean_molecular_mass * gp)).to(u.km)
transiting['signal'] = (2*rp*transiting['H']/transiting['st_rad']**2).decompose()

Apply the selection constraints and prepare for output:

In [35]:
temp_constraints = (equilibrium_temperature < teq_max) & (equilibrium_temperature > teq_min)
rad_constraints = (rp < rad_max) & (rp > rad_min)
gmag_constaints = (g < gaia_mag_limit)
targets = transiting[rad_constraints & temp_constraints & gmag_constaints]#This is the table that will be plotted.

In [36]:
targets.sort('st_j')
targets['r_earth'] = targets['pl_radj'].to(u.R_earth)
targets['teq_print']=np.round(targets['teq'],0)
targets['r_earth_print']=np.round(targets['r_earth'],2)
targets['H_print']=np.round(targets['H'],0)
targets['pl_orbper_print']=np.round(targets['pl_orbper'],2)
targets['dec_print']=np.round(targets['dec'],1)
targets['duration_print']=np.round(targets['pl_trandur']*24.0,1)
targets['signal_print']=np.round(1e6*targets['signal'],2)
targets[['pl_name', 'st_j', 'teq_print', 'r_earth_print', 'pl_bmasse','pl_orbper_print','H_print','signal_print','duration_print','dec','st_spstr']].show_in_notebook()

idx,pl_name,st_j,teq_print,r_earth_print,pl_bmasse,pl_orbper_print,H_print,signal_print,duration_print,dec,st_spstr
,,,K,earthRad,,d,km,,,deg,
0,HD 219134 c,3.981,783.0,1.51,4.36,6.76,25.0,1.66,1.7,57.168354,K3 V
1,HD 21749 c,6.081,705.0,0.9,3.7,7.79,8.0,0.4,2.5,-63.4991,K4.5
2,GJ 143 b,6.081,425.0,2.61,22.7,35.61,41.0,5.92,3.2,-63.4991,K4.5
3,Kepler-444 c,7.244,954.0,0.49,--,4.55,3.0,0.08,--,41.634605,K
4,Kepler-444 d,7.244,860.0,0.53,--,6.19,3.0,0.08,--,41.634605,K
5,Kepler-444 e,7.244,799.0,0.55,--,7.74,3.0,0.09,--,41.634605,K
6,Kepler-444 f,7.244,740.0,0.74,--,9.74,6.0,0.2,--,41.634605,K
7,LTT 1445 A b,7.294,436.0,1.38,2.2,5.36,12.0,5.44,1.4,-16.593372,M3
8,GJ 357 b,7.337,527.0,1.22,1.84,3.93,11.0,3.1,1.5,-21.660797,M2.5 V


In [15]:
#table[['pl_name','pl_orbper']].show_in_notebook() #This outputs the entire table of exoplanets.